In [1]:
import tools
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# import data 
wd = Path.cwd()
data = pd.read_csv(wd.parent/'data'/'data_transformed'/'outage_bunch.csv')

data.head()


C:\Users\paula\anaconda3\envs\dehli\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,division_name,id,area_affected,noofcustomersaffected,capacity,duration,unservedmuduetooutage,remarks,date,end_date,...,dairy,residential,minute_of_day,q,quarter,hourly,year,r_permin,pi,duration_cf
0,alaknanda,1.0,"part off dsidc okhla phase‐1,",312.0,other outage,0:43,0.000005,outage due to\nfault,2019-12-02 00:00:00,2019-12-02 00:00:00,...,0.0,0.0,31.0,3.0,49898.0,16993.0,2019.0,0.116279,0.930233,43.0
1,alaknanda,2.0,"part off indira kalyan vihar okhla phase‐1,",314.0,other outage,0:09,0.000017,outage due to\nfault,2019-12-03 00:00:00,2019-12-03 00:00:00,...,0.0,0.0,30.0,3.0,49924.0,17003.0,2019.0,1.888889,15.111111,9.0
2,alaknanda,3.0,"part off b‐block kalka ji,e‐block kalka ji,f‐b...",303.0,other outage,0:57,0.000633,outage due to fault,2019-12-05 00:00:00,2019-12-05 00:00:00,...,0.0,0.0,19.0,2.0,50008.0,17036.0,2019.0,11.105263,88.842105,57.0
3,alaknanda,4.0,"part off railway yard,",302.0,other outage,0:37,0.000319,outage due to\nfault,2019-12-06 00:00:00,2019-12-06 00:00:00,...,0.0,0.0,21.0,2.0,50072.0,17060.0,2019.0,8.621622,68.972973,37.0
4,alaknanda,5.0,"part off cng station okhla phase‐1,a‐block okh...",140.0,other outage,0:33,0.000413,outage due to fault,2019-12-06 00:00:00,2019-12-06 00:00:00,...,0.0,0.0,30.0,3.0,50063.0,17057.0,2019.0,12.515152,100.121212,33.0


In [7]:
# exchange rate Rupies to USD
rp_usd = 0.014

# define parameters 
phi = 50/60 
phi120 = 100/60



$K(\sigma,\pi, \phi) = (\pi + \phi)^{\alpha / (1+\alpha)} \sigma (1 + 1/\alpha)$

Calculating counterfactuals: now that we have sigma, we can compute any counterfactual cost we want, but choosing for example different parameter values for $\pi$ or $\phi$. The steps are:

1- given the sigmas and our chosen parameter values, calculation the implied xs (from xopt)

2- then given xopt and the parameters chosen we calculate K.

In [8]:
# calculate K 

# method 1: use function tools.K
#counterfactual cost without a fine (phi=0)
data['cost_cf'] = data.apply(lambda row: tools.K(row.duration_cf, sigma = row.sigma, pi = pi, phi=0, phi120=0, alpha = alpha, N = row.noofcustomersaffected), axis = 1)
# what are the right values for phi?
data['cost_real'] = data.apply(lambda row: tools.K(row.duration_minutes,sigma = row.sigma, pi = pi, phi=phi, phi120=phi120, alpha = alpha, N = row.noofcustomersaffected), axis = 1)

#this is wrong
# take xopt and double pi
# phi = 0 ??
data['xopt'] = data.apply(lambda row: tools.xopt(row.sigma, row.pi, 0, alpha, row.noofcustomersaffected), axis = 1)
data['cost_2pi'] = data.apply(lambda row: tools.K(row.xopt, sigma = row.sigma, pi = 2*row.pi, phi=0, phi120=0, alpha = alpha, N = row.noofcustomersaffected), axis = 1)

# get difference
# why do we multiply ba 0.014?
print((data.cost_real.sum() - data.cost_cf.sum())*rp_usd) # 0.014 Rupies to USD
print('doubling pi delta PS:',(data.cost_real.sum() -data.cost_2pi.sum())*rp_usd)

AttributeError: 'Series' object has no attribute 'sigma'

In [ ]:
#print('doubling pi delta PS:',(data.cost_cf_2pi.sum() -data.cost_cf.sum())*0.014)

# Utility of consumer  𝑖  experiencing an outage:

$U_i=\delta T−\pi (T-x)−(p' + \chi)x + FINE $

T is total time (e.g. 24 hours)

Change in consumer surplus:

$ \Delta U_i = (x^{data} - x^{counterfactual})*(\pi-p')$

Since $x^{data} < x^{counterfactual}$ and $𝑝′≥\pi$, and $\chi>0$, $\Delta U_i\geq0 $


Assumptions:

1-  

2- pollution during an outage is only local

Utility of consumer not experiencing an outage:
$U_i = \delta T - p T - \chi \sum_{i} x_{-i} $

In [ ]:
# calculate fine for each outage
data['fine_real'] = data.apply(lambda row: tools.V(phi, phi120, row.duration_minutes, N = row.noofcustomersaffected), axis=1)

data['fine_cf'] = data.apply(lambda row: tools.V(0, 0, row.duration_cf, N = row.noofcustomersaffected), axis=1)

In [ ]:
# total time
start = data.start_date.min()
end = data.end_date.max()

time_diff = end-start
T = time_diff.total_seconds() / 60
print('total time in minutes:',T)
#T = 12*365

# health cost
#chi = 391000000/data.duration_minutes.sum()
chi = 515
print('chi:', chi)

# at outage level
data['pprime'] = data.apply(lambda row: (row.unservedmuduetooutage * 1000000/ row.duration_minutes ) * 21 , axis=1) # is that right?
print('median pprime:', data.pprime.median())

### change in CS: xbunch and xcounter

data['CS_cf'] = data.apply(lambda row: (- pi * (T-row.duration_cf) - row.pprime*row.duration_cf), axis = 1)
##data['CS_cf_nopol'] = data.apply(lambda row: (- pi * (T-row.duration_cf) - pprime*row.duration_cf ), axis = 1)
#data['CS_cf_2pi'] = data.apply(lambda row: (- 2*pi * (T-row.duration_cf) - pprime*row.duration_cf - chi*row.duration_cf, axis = 1)
#data['CS_cf_nopol_2pi'] = data.apply(lambda row: (- 2*pi * (T-row.duration_cf) - pprime*row.duration_cf ), axis = 1)

data['CS_real'] = data.apply(lambda row: (- pi * (T-row.duration_minutes) - row.pprime*row.duration_minutes + row.fine_real), axis = 1)
#data['CS_real_nopol'] = data.apply(lambda row: (- pi * (T-row.duration_minutes) - pprime*row.duration_minutes + row.fine_real), axis = 1)


# get difference
print('CS with pollution', (data.CS_real.sum() - data.CS_cf.sum())*rp_usd) #, 'CS No Pollution', (data.CS_real_nopol.sum() - data.CS_cf_nopol.sum())*0.014,)
#print('CS double PI', (data.CS_cf_2pi.sum() - data.CS_cf.sum())*0.014)

In [ ]:
#back of the envelope calculation (total kwh lost * price difference per kwh)--> corresponds to counterfactual of removing outages completely
print(data.unservedmuduetooutage.sum()*10000000* (7-18))

In [ ]:
## Total change in Welfare 35357207.35

print('change in welfare with pollution is: USD', (data.CS_real.sum() - data.CS_cf.sum() - (data.cost_real.sum() - data.cost_cf.sum()))*rp_usd) #, 'change in welfare without pollution is: USD', (data.CS_real_nopol.sum() - data.CS_cf_nopol.sum() - (data.cost_real.sum() - data.cost_cf.sum()))*0.014)

In [ ]:
print((data.duration_minutes.sum() - data.duration_cf.sum())/91230)